In [1]:
import pandas as pd
import json
import random
import csv
import numpy as np
from datetime import datetime

In [2]:
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline
seaborn.set(rc={'figure.figsize':(15,12)})

In [2]:
def json_to_series(text):
    keys, values = zip(*[item for item in json.loads(text).items()])
    return pd.Series(values, index=keys)

In [ ]:
def summary(input_df,col):
    df = input_df.groupby(col)['transactionRevenue'].count()

    pos_counts = input_df.loc[input_df['transactionRevenue']>0,].groupby(col)['transactionRevenue'].count()

    df = pd.DataFrame({'all':df,'pos':pos_counts})
    df['rate'] = df['pos']/df['all']*100
    df.sort_values(by='rate',ascending=False,inplace=True)
    return df

# Random sample 10000 rows of training set

In [ ]:
filename = ".\\all\\train_v2.csv"

In [ ]:
n = sum(1 for line in open(filename,encoding="utf8")) - 1 

In [ ]:
s = 90000
skip = sorted(random.sample(range(1,1708337+1),1708337-s))
train_top = pd.read_csv(filename,skiprows=skip)

In [ ]:
train_top.to_csv("train_top.csv",index=False)

In [3]:
df = pd.read_csv("train_top.csv")

# Data Cleaning

### Clean ID
sessionId = fullVisitorId_visitId    
visitNumber may be strong indicators since it means the visitor comes back multiple times

In [4]:
ct = df.groupby(by='fullVisitorId')['visitId'].count()
visitn = df.groupby(by='fullVisitorId')['visitNumber'].max()
visitn = pd.concat([visitn,ct],axis=1)

In [5]:
df.drop(['visitNumber','hits','trafficSource'],axis=1,inplace=True)

### Clean Totals

In [6]:
df = pd.concat([df, df['totals'].apply(json_to_series)], axis=1)

In [7]:
df.drop(['totals'],axis=1,inplace=True)

In [8]:
#df['hits'] = df['hits'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['pageviews'] = df['pageviews'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['newVisits'] = df['newVisits'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['visits'] = df['visits'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['bounces'] = df['bounces'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['transactionRevenue'] = df['transactionRevenue'].map(lambda x: int(x) if not pd.isnull(x) else 0)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].map(lambda x: int(x) if not pd.isnull(x) else 0)

In [9]:
df.drop(['totalTransactionRevenue'],axis=1,inplace=True)

### Clean Time Series

In [10]:
df['date'] = pd.to_datetime(df['date'],format="%Y%m%d")
df['week'] = df['date'].map(lambda x: x.isocalendar()[1])
df['day_of_week'] = df['date'].map(lambda x: x.isocalendar()[2])

In [11]:
df.drop(['date'],axis=1,inplace=True)

In [12]:
df["visitStartTime"] = df["visitStartTime"].map(lambda x:datetime.utcfromtimestamp(x).hour)

### Clean Location

In [13]:
df = pd.concat([df, df['geoNetwork'].apply(json_to_series)], axis=1)

In [14]:
df.drop(['geoNetwork'],axis=1,inplace=True)

### Clean Device

In [15]:
df = pd.concat([df, df['device'].apply(json_to_series)], axis=1)

In [16]:
df.drop(['device'],axis=1,inplace=True)

### Clean customDimensions

In [17]:
df['customDimensions'] = df['customDimensions'].map(lambda x: x[1:-1])

In [18]:
df['customDimensions'] = df['customDimensions'].map(lambda x: x.replace("'","\""))

In [19]:
df.loc[df['customDimensions']=="",'customDimensions'] = "{\"index\":\"NaN\",\"value\":\"NaN\"}"

In [20]:
df = pd.concat([df, df['customDimensions'].apply(json_to_series)], axis=1)

In [21]:
df.drop(['customDimensions'],axis=1,inplace=True)

# Feature Cleaning

In [197]:
ID = ["fullVisitorId","visitId"]

#### Drop uninformative categorical varibles

In [22]:
for col in df.columns:
    cn = df[col].value_counts()
    if cn.shape[0]==1:
        df.drop(col,axis=1,inplace=True)

#### Drop variables with too many missing values

In [23]:
df.replace("not available in demo dataset",np.NaN,inplace=True)

In [24]:
df.replace("NaN",np.NaN,inplace=True)

In [25]:
df.replace("(not set)",np.NaN,inplace=True)

In [26]:
na_df = np.sum(df.isna(),axis=0)/df.shape[0]
na_df[na_df>0.1]

timeOnSite           0.513856
sessionQualityDim    0.488111
transactions         0.989322
region               0.577833
metro                0.773100
city                 0.588144
networkDomain        0.291833
index                0.197367
value                0.197367
dtype: float64

In [27]:
df.drop(['index',"isMobile","transactions","metro"],axis=1,inplace=True)

# Fit Random Forest

In [28]:
df.to_csv("train.csv",index=False)

In [29]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.152-b16, mixed mode)
  Starting server from C:\Users\XuL\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\XuL\AppData\Local\Temp\tmp552rek29
  JVM stdout: C:\Users\XuL\AppData\Local\Temp\tmp552rek29\h2o_XuL_started_from_python.out
  JVM stderr: C:\Users\XuL\AppData\Local\Temp\tmp552rek29\h2o_XuL_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,20 days
H2O cluster name:,H2O_from_python_XuL_y2ik6k
H2O cluster total nodes:,1
H2O cluster free memory:,3.540 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# Data Analysis and Visualization